Импортируем pandas

In [3425]:
import pandas as pd

Импортируем данные о стоимости машин

In [3426]:
car_price = pd.read_csv('https://raw.githubusercontent.com/rmadan16/LinearRegression_CarPrice/refs/heads/master/CarPrice_Assignment.csv')

Посмотрим, какие столбцы присутствуют в таблице с данными

In [3427]:
car_price.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

Определим признаки (X) и зависимые переменные (y)

Попробуем для начала взять один численный и один категориальный признак, например, количество лошадиных сил и тип кузова

In [3428]:
y = car_price['price']
X = car_price[['carbody', 'horsepower']]
X

,carbody,horsepower
0,convertible,111
1,convertible,111
2,hatchback,154
3,sedan,102
4,sedan,115
...,...,...
200,sedan,114
201,sedan,160
202,sedan,134
203,sedan,106


Теперь нужно преобразовать категориальный признак. Импользуем One-Hot Encoding, так как категории не имеют определённого порядка

In [3429]:
from sklearn.preprocessing import OneHotEncoder

In [3430]:
encoder = OneHotEncoder()

In [3431]:
encoded_features = encoder.fit_transform(X[['carbody']]).toarray()
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['carbody']))
X = pd.concat([X, encoded_df], axis=1).drop('carbody', axis=1)
X

,horsepower,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon
0,111,1.0,0.0,0.0,0.0,0.0
1,111,1.0,0.0,0.0,0.0,0.0
2,154,0.0,0.0,1.0,0.0,0.0
3,102,0.0,0.0,0.0,1.0,0.0
4,115,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
200,114,0.0,0.0,0.0,1.0,0.0
201,160,0.0,0.0,0.0,1.0,0.0
202,134,0.0,0.0,0.0,1.0,0.0
203,106,0.0,0.0,0.0,1.0,0.0


Теперь нужно разделить данные на обучающую и тестовую выборки

In [3432]:
from sklearn.model_selection import train_test_split
initital = 21
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=initital)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((143, 6), (62, 6), (143,), (62,))

Далее используем модель линейной регрессии

In [3433]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

Обучение модели

In [3434]:
model.fit(X_train, y_train)

LinearRegression()

Свободный член

In [3435]:
model.intercept_

-2320.337286063732

Коэффициенты (веса)

In [3436]:
model.feature_names_in_

array(['horsepower', 'carbody_convertible', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon'],
      dtype=object)

In [3437]:
model.coef_

array([  167.60212054,  2143.22474827,  3449.96083454, -3971.16706924,
        -578.65611542, -1043.36239814])

Попытка предсказаний на тестовой выборке

In [3438]:
y_pred = model.predict(X_test)

Импортируем метрики, чтобы узнать, что в итоге получилось

In [3439]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [3440]:
mean_absolute_error(y_test, y_pred)

3232.7207820177714

In [3441]:
mean_absolute_percentage_error(y_test, y_pred)

0.2615057143981352

In [3442]:
mean_squared_error(y_test, y_pred)

19621208.947314292

Получили ошибку в 26%, что довольно много. Попробуем добавить признаков и обучить снова

Добавим информацию о размерах автомобиля и типе топлива

In [3443]:
X = pd.concat([X, car_price[['carlength', 'carwidth', 'carheight']]], axis=1)
encoded_features = encoder.fit_transform(car_price[['fueltype']]).toarray()
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['fueltype']))
X = pd.concat([X, encoded_df], axis=1)
X

,horsepower,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,carlength,carwidth,carheight,fueltype_diesel,fueltype_gas
0,111,1.0,0.0,0.0,0.0,0.0,168.8,64.1,48.8,0.0,1.0
1,111,1.0,0.0,0.0,0.0,0.0,168.8,64.1,48.8,0.0,1.0
2,154,0.0,0.0,1.0,0.0,0.0,171.2,65.5,52.4,0.0,1.0
3,102,0.0,0.0,0.0,1.0,0.0,176.6,66.2,54.3,0.0,1.0
4,115,0.0,0.0,0.0,1.0,0.0,176.6,66.4,54.3,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
200,114,0.0,0.0,0.0,1.0,0.0,188.8,68.9,55.5,0.0,1.0
201,160,0.0,0.0,0.0,1.0,0.0,188.8,68.8,55.5,0.0,1.0
202,134,0.0,0.0,0.0,1.0,0.0,188.8,68.9,55.5,0.0,1.0
203,106,0.0,0.0,0.0,1.0,0.0,188.8,68.9,55.5,1.0,0.0


Повторное раздление данных

In [3444]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=initital)

Повторное обучение

In [3445]:
model.fit(X_train, y_train)

LinearRegression()

In [3446]:
model.intercept_

-101689.48130035019

In [3447]:
coef_df = pd.DataFrame({
    'feature': model.feature_names_in_,
    'coef': model.coef_
})
coef_df

,feature,coef
0,horsepower,125.659574
1,carbody_convertible,4476.409655
2,carbody_hardtop,2789.765472
3,carbody_hatchback,-2684.958969
4,carbody_sedan,-1269.827230
5,carbody_wagon,-3311.388928
6,carlength,-17.788927
7,carwidth,1408.002218
8,carheight,279.481767
9,fueltype_diesel,1126.826728


In [3448]:
y_pred = model.predict(X_test)

In [3449]:
mean_absolute_error(y_test, y_pred)

3033.022763227723

In [3450]:
mean_absolute_percentage_error(y_test, y_pred)

0.25316199382483495

In [3451]:
mean_squared_error(y_test, y_pred)

14738018.803218946

Ошибка немного уменьшилась и также видно, что нектороые данные имеют заметно меньшее влияние, чем другие. Например, лошадиные силы, длина и высота машины. Попробуем избавиться от них

In [3452]:
X = X.drop(['horsepower', 'carlength', 'carheight'], axis=1)

Обучим ещё раз, посмотрим, как это влияет на точность предсказаний

In [3453]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=initital)
model.fit(X_train, y_train)
coef_df = pd.DataFrame({
    'feature': model.feature_names_in_,
    'coef': model.coef_
})
coef_df

,feature,coef
0,carbody_convertible,6611.695063
1,carbody_hardtop,4621.707659
2,carbody_hatchback,-3974.596566
3,carbody_sedan,-1939.058884
4,carbody_wagon,-5319.747272
5,carwidth,2928.268460
6,fueltype_diesel,-1210.260480
7,fueltype_gas,1210.260480


In [3454]:
y_pred = model.predict(X_test)

In [3455]:
mean_absolute_error(y_test, y_pred)

3357.156933945885

In [3456]:
mean_absolute_percentage_error(y_test, y_pred)

0.2854808446003196

In [3457]:
mean_squared_error(y_test, y_pred)

22094941.73406909

После избавления от незначимых данных ошибка незначительно выросла. Попробуем найти что-то более значимое. Попробуем использовать сразу все данные (кроме названия) и посмотрим на коэффициенты.

In [3458]:
X = car_price[['wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]

In [3459]:
encoded_features = encoder.fit_transform(car_price[['fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem']]).toarray()
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())
X = pd.concat([X, encoded_df], axis=1)
X

,wheelbase,carlength,carwidth,carheight,curbweight,boreratio,stroke,compressionratio,horsepower,peakrpm,...,enginesize_308,enginesize_326,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,88.6,168.8,64.1,48.8,2548,3.47,2.68,9.0,111,5000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,88.6,168.8,64.1,48.8,2548,3.47,2.68,9.0,111,5000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,94.5,171.2,65.5,52.4,2823,2.68,3.47,9.0,154,5000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,99.8,176.6,66.2,54.3,2337,3.19,3.40,10.0,102,5500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,99.4,176.6,66.4,54.3,2824,3.19,3.40,8.0,115,5500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,109.1,188.8,68.9,55.5,2952,3.78,3.15,9.5,114,5400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
201,109.1,188.8,68.8,55.5,3049,3.78,3.15,8.7,160,5300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
202,109.1,188.8,68.9,55.5,3012,3.58,2.87,8.8,134,5500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
203,109.1,188.8,68.9,55.5,3217,3.01,3.40,23.0,106,4800,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3460]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=initital)
model.fit(X_train, y_train)

LinearRegression()

In [3461]:
model.intercept_

-41242.932218064016

In [3462]:
coef_df = pd.DataFrame({
    'feature': model.feature_names_in_,
    'coef': model.coef_
})
for index, row in coef_df.iterrows():
    print(f"{row['feature']}: {row['coef']}")

wheelbase: -2.423392697910863
carlength: -103.97570706617924
carwidth: 691.2051885330083
carheight: -193.6883231132314
curbweight: 14.254508625969606
boreratio: 718.0431009480349
stroke: -2501.21742530932
compressionratio: 1436.332546671241
horsepower: -7.159938691072739
peakrpm: -0.7180447220539463
citympg: -37.68474380562043
highwaympg: 24.730282240608176
fueltype_diesel: -6951.559501242762
fueltype_gas: 6951.559501243209
aspiration_std: -1303.9020877874466
aspiration_turbo: 1303.9020877898697
doornumber_four: -148.78623070297738
doornumber_two: 148.78623070330286
carbody_convertible: 1705.1966449272009
carbody_hardtop: -371.4687265720195
carbody_hatchback: -341.57749628544116
carbody_sedan: 212.2231810280573
carbody_wagon: -1204.3736030963937
drivewheel_4wd: -151.99211530144476
drivewheel_fwd: -2.7069203750370434
drivewheel_rwd: 154.6990356770193
enginelocation_front: -7460.987495597474
enginelocation_rear: 7460.987495597511
enginetype_dohc: 5229.9045195038225
enginetype_dohcv: 1.45

In [3463]:
y_pred = model.predict(X_test)

In [3464]:
mean_absolute_error(y_test, y_pred)

1657.941296756953

In [3465]:
mean_absolute_percentage_error(y_test, y_pred)

0.1509055622369788

In [3466]:
mean_squared_error(y_test, y_pred)

5756394.816303857

Получена на данный момент наибольшая точность, но используется слишком много данных. Попробуем снизить количество признаков.

In [3467]:
mean_coef = coef_df['coef'].abs().mean()
mean_coef

2492.8021948543087

In [3468]:
filtered_coef = coef_df[coef_df['coef'].abs() >= mean_coef]
filtered_coef

,feature,coef
6,stroke,-2501.217425
12,fueltype_diesel,-6951.559501
13,fueltype_gas,6951.559501
26,enginelocation_front,-7460.987496
27,enginelocation_rear,7460.987496
28,enginetype_dohc,5229.904520
30,enginetype_l,-4370.284226
31,enginetype_ohc,4026.824105
32,enginetype_ohcf,-4954.058048
35,cylindernumber_eight,6463.711826


In [3469]:
X = X[filtered_coef['feature']]
X

,stroke,fueltype_diesel,fueltype_gas,enginelocation_front,enginelocation_rear,enginetype_dohc,enginetype_l,enginetype_ohc,enginetype_ohcf,cylindernumber_eight,...,enginesize_171,enginesize_181,enginesize_194,enginesize_209,enginesize_234,enginesize_304,enginesize_308,enginesize_326,fuelsystem_1bbl,fuelsystem_idi
0,2.68,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.68,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.47,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.40,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.40,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,3.15,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,3.15,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,2.87,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203,3.40,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Из набора признаков были удалены все те, чьи веса по модулю меньше среднего значения. Это сократило число признаков почти в три раза. Посмотрим, как это повлияет на модель

In [3470]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=initital)
model.fit(X_train, y_train)

LinearRegression()

In [3471]:
model.intercept_

11006.284556132134

In [3472]:
coef_df = pd.DataFrame({
    'feature': model.feature_names_in_,
    'coef': model.coef_
})
coef_df

,feature,coef
0,stroke,2314.171015
1,fueltype_diesel,1015.351377
2,fueltype_gas,-1015.351377
3,enginelocation_front,-4503.559832
4,enginelocation_rear,4503.559832
5,enginetype_dohc,18762.533521
6,enginetype_l,22623.136871
7,enginetype_ohc,16033.655279
8,enginetype_ohcf,9068.851213
9,cylindernumber_eight,20459.751004


In [3473]:
y_pred = model.predict(X_test)

In [3474]:
mean_absolute_error(y_test, y_pred)

1997.836825009927

In [3475]:
mean_absolute_percentage_error(y_test, y_pred)

0.17778911493149224

In [3476]:
mean_squared_error(y_test, y_pred)

7405383.75309055

Удалось значительно оптимизировать модель при незначительной потере в точности. Точность всё ещё далека от идеала. Возможно, требуется увеличение размера выборки или использование другой модели.